In [108]:
#!/usr/bin/python3
# Install TensorFlow
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import activations
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate, Input, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
import tensorflow.keras.backend as K
from sklearn.preprocessing import StandardScaler
from joblib import dump, load

import numpy as np
import matplotlib.pyplot as plt
# import autokeras as ak
import os 
import sys
import time
import importlib
import logging
from tqdm import tqdm

importlib.reload(logging)
logging.basicConfig(level = logging.INFO)

# limit GPU memory
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    print(e)

logging.info("numpy Version is {}".format(np.__version__))
# logging.info("autokeras Version is {}".format(ak.__version__))
logging.info("tensorflow Version is {}".format(tf.keras.__version__))
logging.info("\n")

!nvidia-smi

INFO:root:numpy Version is 1.19.5
INFO:root:tensorflow Version is 2.4.0
INFO:root:



1 Physical GPUs, 1 Logical GPU
Thu Nov  4 11:44:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM-80GB       On   | 00000000:0B:00.0 Off |                    0 |
| N/A   33C    P0    69W / 400W |    568MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                         

In [2]:
try:

    """
    Define Name
    """
    experiment = "dune"
    
    logging.info("experiment: {}".format(experiment))

except:
    print("********* Please Check Input Argunment *********")
    print("********* Usage: python3 poisson_learning_study_v2_to_5GeV.py experiment physics_parameter *********")
    sys.exit(1)

INFO:root:experiment: dune


In [134]:
%%time
"""
Load Data
"""
#======================================================#
training_data = np.load("../Data/n1000000_0910_all_flat.npz")
#======================================================#


"""
Stack Data
"""
#======================================================#
data_all = np.column_stack([training_data["ve_"+str(experiment)][:,:36], training_data["vu_"+str(experiment)][:,:36], training_data["vebar_"+str(experiment)][:,:36], training_data["vubar_"+str(experiment)][:,:36]])
data_all = data_all.reshape(len(data_all),4,36)
data_all = np.array([ element.T for element in data_all])


target = np.column_stack( [training_data["delta"], training_data["theta23"]])
target = target/180*np.pi 

x_train = data_all[:900000]
y_train = target[:900000]
y_train = np.column_stack([np.sin(y_train[:,0]), np.cos(y_train[:,0]), np.sin(y_train[:,1]), np.cos(y_train[:,1])]) # 10/26 modified

x_test = data_all[900000:]
y_test = target[900000:]
y_test = np.column_stack([np.sin(y_test[:,0]), np.cos(y_test[:,0]), np.sin(y_test[:,1]), np.cos(y_test[:,1])]) # 10/26 modified

logging.info("X train shape: {}".format(x_train.shape))
logging.info("X test shape: {}".format(x_test.shape))
logging.info("Y train shape: {}".format(y_train.shape))
logging.info("Y test shape: {}".format(y_test.shape))




INFO:root:X train shape: (900000, 36, 4)
INFO:root:X test shape: (100000, 36, 4)
INFO:root:Y train shape: (900000, 4)
INFO:root:Y test shape: (100000, 4)


CPU times: user 3.46 s, sys: 652 ms, total: 4.12 s
Wall time: 4.11 s


In [135]:
def Regression_Model(trig=False):
    #Ref: https://towardsdatascience.com/can-machine-learn-the-concept-of-sine-4047dced3f11
    inputs = Input(shape=(36,4),name = 'input')
    conv1d_1 = Conv1D(filters=1, kernel_size=10,strides=1, activation='relu', name = 'conv1d_1')(inputs)
    conv1d_2 = Conv1D(filters=1, kernel_size=10,strides=1, activation='relu', name = 'conv1d_2')(conv1d_1)
    flatten_1 = Flatten(name = "flatten_1")(conv1d_2)
    dense_1 = Dense(512, activation='relu', name = 'dense_1')(flatten_1)
    dense_2 = Dense(256, activation='relu', name = 'dense_2')(dense_1)
    dense_3 = Dense(256, activation='relu', name = 'dense_3')(dense_2)
    dense_4 = Dense(128, activation='relu', name = 'dense_4')(dense_3)
    dense_5 = Dense(128, activation='relu', name = 'dense_5')(dense_4)
    if trig == False:
        out1 = Dense(1, activation='linear', name = "output1")(dense_5)
#         out2 = Dense(1, activation='linear', name = "output2")(dense_5)
    elif trig == True:
        out1 = Dense(4, activation='linear', name = "output1")(dense_5)
#         out2 = Dense(2, activation='linear', name = "output2")(dense_5)

#     regression_model = Model(inputs=inputs, outputs=[out1,out2], name = 'Model')
    regression_model = Model(inputs=inputs, outputs=out1, name = 'Model')



#     model_opt = keras.optimizers.Adadelta()
    model_opt = keras.optimizers.Adam()

    regression_model.compile(loss="mean_squared_error",
                       optimizer=model_opt,
                       metrics=["mse","mae"])

    return regression_model

In [136]:
model = Regression_Model(trig=True) 
model.summary()

Model: "Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 36, 4)]           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 27, 1)             41        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 18, 1)             11        
_________________________________________________________________
flatten_1 (Flatten)          (None, 18)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               9728      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792 

In [137]:
model.fit( x_train,
           y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=1,
           verbose=1,
           shuffle = True,
#                callbacks=check_list
             )

12657/12657 [==============================] - 43s 2ms/step - loss: 0.1128 - mse: 0.1128 - mae: 0.1601 - val_loss: 7.0185e-04 - val_mse: 7.0185e-04 - val_mae: 0.0186


In [138]:
prediction = model.predict(x_test)

In [139]:
prediction[:2]

array([[ 0.97221226,  0.22528848,  0.6468325 ,  0.76070523],
       [ 0.89982945, -0.45764026,  0.62512064,  0.7744713 ]],
      dtype=float32)

In [140]:
y_test[:2]

array([[ 0.96604588,  0.25837058,  0.65322108,  0.75716723],
       [ 0.92350506, -0.38358623,  0.63668675,  0.77112255]])

In [142]:
np.arctan2(0.97221226,  0.22528848)

1.3430876929361821

In [141]:
np.arctan2(y_test[0,:])

1.309461195224224

In [149]:
np.arctan2(prediction[:,2],prediction[:,3])# - np.arctan2(y_test[:,2:4])

array([0.7046719 , 0.6790903 , 0.7609129 , ..., 0.71921605, 0.8188449 ,
       0.73160243], dtype=float32)

In [153]:
np.mean((np.arctan2(prediction[:,2],prediction[:,3])- np.arctan2(y_test[:,2],y_test[:,3]))**2)

0.0004055870111748446